- This notebook loads a dataframe stored as a parquet file containing track information.
- It then breaks the dataframe by user, and then further into size of 100 tracks/rows.
- track id is used to request track analysis from spotify web api
- tempo and segment information is saved as dataframes in parquet to access later

In [1]:
%load_ext autoreload

In [2]:
from cap_package import SpotipyCollectPub as scp
from datetime import datetime
import datetime as dt
import demoji
from dotenv import load_dotenv
from IPython.utils.text import columnize
import os
import pandas as pd
from pathlib import Path
import re
import spotipy

... OK (Got response in 0.21 seconds)
Writing emoji data to C:\Users\Administrator\.demoji\codes.json ...
... OK
... OK (Got response in 0.19 seconds)
Writing emoji data to C:\Users\Administrator\.demoji\codes.json ...
... OK


In [3]:
%autoreload 2

In [4]:
def disp_col(list_):
    
    # import -> from IPython.utils.text import columnize
    l = list(map(lambda x:repr(x)+',', list_))
    print(columnize(l, displaywidth=120))

In [5]:
def timer(start_time=None):
    if start_time is not None:
        return (datetime.now() - start_time).total_seconds()
    else:
        start_time = datetime.now()
        return start_time

In [6]:
disp_col(dir(scp))

'SpotifyClientCredentials',  '__loader__',   'filterby_keyword',      'get_tracks_df',   'spotipy',            
'__builtins__',              '__name__',     'get_artist_name',       'json_normalize',  'spotipy_client_cred',
'__cached__',                '__package__',  'get_df_analysis',       'pd',              'uri_to_id',          
'__doc__',                   '__spec__',     'get_public_playlists',  're',              'user_analysis',      
'__file__',                  'demoji',       'get_tracks',            'sc',              'user_plid_pair',     



In [7]:
load_dotenv()
SPOTIPY_CLIENT_ID = os.getenv('CLIENT_ID')
SPOTIPY_CLIENT_SECRET = os.getenv('CLIENT_SECRET')
path = os.getenv('PATH_DATASET2')

In [8]:
sp = scp.spotipy_client_cred(client_id=SPOTIPY_CLIENT_ID, client_secret=SPOTIPY_CLIENT_SECRET)

In [9]:
path_ = Path(path).joinpath('musiclabels_tracks_.parquet')
tracks_df = pd.read_parquet(path_)
tracks_df.tail()

,name,id,artists_name,user
18396,Volume One - Free State vs Dirt Devils Remix,03NSRR3kgp4oOKqGtTEjOZ,Anjunabeats,Anjunabeats
18397,Volume One - Anjuna Deep Mix,4qKcK7oKhB2oMF1Ny2T4Rr,Anjunabeats,Anjunabeats
18398,Volume One - Original Mix,1ZfyvweqqR3csr8K4ACjt4,Anjunabeats,Anjunabeats
18399,Volume One - Tease Dub,4GYyq7cPaTjyynNWpPkC9P,Anjunabeats,Anjunabeats
18400,Won't Sleep Tonight - Moody Dub Mix,3G2fWbQSseRglXGRhC99Jm,Super8 & Tab,Anjunabeats


In [10]:
users = list(tracks_df['user'].unique())
print(users)

['ChillBeatsMusic', 'anjunadeep', 'spinninrecordsofficial', 'enhanced_music', 'emergentmusic', 'armadamusicofficial', 'spotify', 'silkmusic', 'Anjunabeats']


In [11]:
tracksdf_byuser = [tracks_df[tracks_df['user'] == u] for u in users]

In [12]:
tracksdf_chunks = []
n = 100 # chunk size
for df in tracksdf_byuser:
    
    list_df = [df[i:i+n] for i in range(0,df.shape[0],n)]
    tracksdf_chunks.append(list_df)
    
print(' There are {} users in our list.'.format(len(tracksdf_byuser)))
for u,l in zip(users, tracksdf_chunks):
    
    print('user : {} , # of dfs : {}'.format(u, len(l)))

 There are 9 users in our list.
user : ChillBeatsMusic , # of dfs : 3
user : anjunadeep , # of dfs : 19
user : spinninrecordsofficial , # of dfs : 33
user : enhanced_music , # of dfs : 2
user : emergentmusic , # of dfs : 3
user : armadamusicofficial , # of dfs : 58
user : spotify , # of dfs : 45
user : silkmusic , # of dfs : 5
user : Anjunabeats , # of dfs : 19


The cell below will do the following :
- for each user go through all dataframes
> request track analysis for all tracks in a datframe
>> save tempo and segment information as parquet to load later

In [ ]:
for u, user_dfs in zip(users, tracksdf_chunks): 
    
    for i in range(len(used_dfs)):
        
        scp.user_analysis(sp, user=u, df=user_dfs[i], path=path, fn=i)
        